In [1]:
from yahoo_finance import Share
import pandas as pd
import matplotlib as plt
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def get_Research(soup):
    research = ["NA","NA","NA"]
    x = soup.findAll("td",recursive=True)
    y = 0
    for i in range(len(x)):

        if re.search("Research",str(x[i])):
            y +=1
            if y > 1:
                research = x[i+1:i+4]

    z = [re.sub(r'[A-Za-z\<\>\=\"\/\&\;\,]',"",str(i)).strip() for i in research]
    return z

def pos_neg(x):
    if re.search(r'[\(]',x):
        return re.sub(r'[\)]',"",re.sub(r'[\(]',"-",x))
    else:
        return x

def get_cf(soup):
    Total_CF = [np.NaN,np.NaN,np.NaN]
    x = soup.findAll("td",recursive=True)
    for i in range(len(x)):
        if re.search("Total Cash Flow From Operating Activities",str(x[i])):

            Total_CF = x[i+1:i+4]

    z = [pos_neg(re.sub(r'[A-Za-z\<\>\=\"\/\&\;\,\\]',"",str(i)).strip()) for i in Total_CF]
    if len(z[0]) < 1 :
        z[0] = np.NaN
    if len(z[1]) < 1 :
        z[1] = np.NaN
    if len(z[2]) < 1 :
        z[2] = np.NaN
    

    return z


def get_expend(soup):
    Capital_Expend = ["NA","NA","NA"]
    x = soup.findAll("td",recursive=True)
    for i in range(len(x)):

        if re.search("Capital Expenditures",str(x[i])):
            Capital_Expend = x[i+1:i+4]

    z = [pos_neg(re.sub(r'[A-Za-z\<\>\=\"\/\&\;\,\\]',"",str(i)).strip()) for i in Capital_Expend]
    if len(z[0]) < 1 :
        z[0] = np.NaN
    if len(z[1]) < 1 :
        z[1] = np.NaN
    if len(z[2]) < 1 :
        z[2] = np.NaN
    return z

def get_OutstandingShare(soup):
    Share_out = ""
    x = soup.findAll("td",recursive=True)
    for i in range(len(x)):
        if re.search("Shares",str(x[i])):

            Share_out = x[i+1]
    #print(Share_out)
    if len(re.findall(r'[A-Z]',str(Share_out))) > 0:
            factor = dictunit[re.findall(r'[A-Z]',str(Share_out))[0]]
    else:
        factor = 1.0
    try:
        #print(str(factor))

        z = float(pos_neg(re.sub(r'[A-Za-z\<\>\=\"\/\&\;\,\\]',"",str(Share_out)).strip()))*factor

    except:
        z = np.NAN

    return(z)

In [13]:
choices = {0:"S&P500", 1:"QNET"}
mychoice = 0 ### Choose Between the two.

if mychoice == 0:
    stocks = pd.read_csv("SP500.csv")
    stocks = stocks[[0,1]]
else:
    stocks = pd.read_csv("QNET_SYMBOLS.csv")
    stocks = stocks[[0,1]]
dictunit = {'B':1000000000,'M':1000000,'':1}

In [14]:
dict_final = {}
for i in stocks.Symbol:
    print(i)
    r = requests.get('http://finance.google.com/finance', params = {'q':i,'fstype':'ii'})
    html_doc = r.text
    #print("html_doc",len(html_doc))
    soup = BeautifulSoup(html_doc,'html.parser')
    Research = get_Research(soup)
    
    
    r.close()
    r1 = requests.get("http://finance.yahoo.com/q/cf?",params={'s':i,'':'annual'})
    html_doc1 = r1.text
    soup1 = BeautifulSoup(html_doc1,'html.parser')
    #print(len(Research))
    r1.close()
    r2 = requests.get("http://finance.yahoo.com/d/quotes.csv?",params = {'s':i,'f' : 'j2'})
    html_doc2 = r2.text
    soup2 = BeautifulSoup(html_doc2,'html.parser')
    r2.close()
    
    
    r3 = requests.get("http://www.google.com/finance",params= {'q':i})
    html_doc3 = r3.text
    soup3 = BeautifulSoup(html_doc3,'html.parser')
    r3.close()
    QNET = Share(i)
    temp_list_yahoo = [QNET.get_price_earnings_ratio(),
                    QNET.get_price_earnings_growth_ratio(),
                    QNET.get_price_sales(),
                    QNET.get_price_book()]
    
    temp_cap = QNET.get_market_cap()
    if type(temp_cap) == str:
        temp_market_cap_num = float(re.findall(r'[0-9\.]*',temp_cap)[0])
        factor = dictunit[re.findall(r'[A-Z]',temp_cap)[0]]
        temp_market_cap = temp_market_cap_num * factor

    else:
        
        temp_market_cap = np.NaN
    
    price_to_research = []
    change_percent = []
    try:
        for j in Research:
            price_to_research.append(temp_market_cap/(1000000*float(j)))
            
    except:
        price_to_research = ["NA","NA","NA"]
    
    try:
        change_percent = [((float(j) / float(Research[2]))-1)*100 for j in Research]
        change_percent.pop()
        change_percent_avg = round(np.average(change_percent),2)
        
    except:
        change_percent_avg = np.NaN
        
    #########Calculating the CF-Yield
    #get_cf(soup1)
    s1 = np.array(get_cf(soup1))
    #print(s1)
    try:
        s1 = s1.astype(float)*(1000)
    except:
        s1 = s1[:-1]
        s1 = s1.astype(float)
        s1 = np.append(s1,np.NAN)
    
    s2 = np.array(get_expend(soup1))
    #print(s2)
    if '-' in s2:
        s2 = [np.NAN,np.NAN,np.NAN]
    else:
        s2 = s2.astype(float)*(1000)
    
    d1 = get_OutstandingShare(soup3)
    #print(float(QNET.get_price()))
    #print(type(d1))
    FCFperShare = (s1+s2)/d1
    
    myprice = QNET.get_price()
    #print(myprice)
    #print(type(myprice))
    if type(myprice) == str:
        FCFYield = (100 * FCFperShare)/(float(QNET.get_price()))
        FCFYield = FCFYield.tolist()
    else:
        FCFYield = [np.NaN]
    
    #print(type(FCFYield))
    #print(FCFYield)

    temp_list = temp_list_yahoo + Research + price_to_research +[change_percent_avg] +[FCFYield[0]]
    #print(temp_list)
    #temp_list = temp_list.append(FCFYield)
    #if not len(temp_list) == 11: print(i,len(temp_list))
    dict_final[i] = temp_list

    

MMM
ABT
ABBV
ACN
ATVI
ADBE
ADT
AAP
AES
AET
AMG
AFL
A
GAS
APD
ARG
AKAM
AA
ALXN
ALLE
AGN
ADS
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AMP
ABC
AME
AMGN
APH
APC
ADI
ANTM
AON
APA
AIV
AAPL
AMAT
ADM
AIZ
T
ADSK
ADP
AN
AZO
AVGO
AVB
AVY
BHI
BLL
BAC
BCR
BXLT
BAX
BBT
BDX
BBBY
BRK-B
BBY
BIIB
BLK
HRB
BA
BWA
BXP
BSX
BMY
BF-B
CHRW
CA
CVC
COG
CAM
CPB
COF
CAH
KMX
CCL
CAT
CBG
CBS
CELG
CNP
CTL
CERN
CF
SCHW
CHK
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CME
CMS
COH
CCE
CTSH
CL
CPGX
CMCSA
CMA
CAG
CXO
COP
CNX
ED
STZ
GLW
COST
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DLPH
DAL
XRAY
DVN
DO
DFS
DISCA
DISCK
DG
DLTR
D
DOV
DOW
DPS
DTE
DD
DUK
DNB
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMC
EMR
ENDP
ESV
ETR
EOG
EQT
EFX
EQIX
EQR
ESS
EL
ES
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FISV
FLIR
FLS
FLR
FMC
FTI
F
BEN
FCX
FTR
GME
GPS
GRMN
GD
GE
GGP
GIS
GM
GPC
GILD
GS
GT
GWW
HAL
HBI
HOG
HAR
HRS
HIG
HAS
HCA
HCP
HP
HSIC
HES
HPE
HD
HON
HRL
HST
HPQ
HUM
HBAN
ITW
ILMN
IR
INTC
ICE
IBM
IP
IPG
IFF
INTU
ISRG
IVZ

C:\Users\Shobeir\Anaconda3\lib\site-packages\ipykernel\__main__.py:73: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [15]:
final_table = pd.DataFrame(dict_final).T

In [16]:
final_table.reset_index(level=0, inplace=True)


In [17]:
final_table.columns = ["Symbol","PE","PE to growth",
                       "Price to sales","Price to book",
                       "Research expense 2015","Research expense 2014","Research expense 2013",
                       "Price to research 2015","Price to research 2014","Price to research 2013",
                       "Percent change per annum","Cash Flow Yield"]

In [18]:
final_table.head()

,Symbol,PE,PE to growth,Price to sales,Price to book,Research expense 2015,Research expense 2014,Research expense 2013,Price to research 2015,Price to research 2014,Price to research 2013,Percent change per annum,Cash Flow Yield
0,A,29.67,2.04,3.31,3.31,326.00,353.00,337.00,41.1656,38.017,39.822,0.74,NaN
1,AA,None,2.33,0.58,1.09,238.00,218.00,192.00,55.2941,60.367,68.5417,18.75,3.06564
2,AAL,3.70,-0.51,0.61,4.56,-,-,-,NA,NA,NA,NaN,0.171107
3,AAP,25.19,1.45,1.21,4.79,-,-,-,NA,NA,NA,NaN,3.83751
4,AAPL,11.69,1.04,2.65,4.85,8067.00,6041.00,4475.00,75.5014,100.823,136.105,57.63,11.5055


In [19]:
final_table.to_csv("final_table_SP500.csv",sep=",",na_rep="")